In [1]:
import requests
from bs4 import BeautifulSoup as Soup
import brotli
import pandas as pd


In [2]:
url = "http://fr.indeed.com/emplois?q=data+engineer&l=Paris&radius=10"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Connection": "keep-alive",
    "Accept-Language": "en-US,en;q=0.9,lt;q=0.8,et;q=0.7,de;q=0.6",
}

In [18]:
df = pd.DataFrame({}, columns=["title", "company", "city", "min_salary", "max_salary", "frequency"])

resp = requests.get(url, headers=headers)
# Détecter l'encodage spécifié dans les headers
content_type = resp.headers.get('Content-Type', '')
print(content_type)
encoding = 'utf-8'  # Encodage par défaut
if 'charset=' in content_type:
    encoding = content_type.split('charset=')[-1]
# Décompresser le contenu si encodé en Brotli
if resp.headers.get('Content-Encoding') == 'br':
    try:
        decompressed_content = brotli.decompress(resp.content)
        text = decompressed_content.decode(encoding)
    except Exception as e:
        print("Error decompressing content:", e)
        text = resp.text
else:
    text = resp.text  # Utiliser la méthode text qui gère automatiquement l'encodage
# Parser le contenu HTML avec BeautifulSoup
soup = Soup(text, "html.parser")

contents = soup.find_all("li", {"class": "css-5lfssm eu4oa1w0"})

for element in contents[:-1]:
    element_dict={}
    head = element.find("h2", {"class": "jobTitle css-198pbd eu4oa1w0"})
    location = element.find("div", {"class": "company_location css-17fky0v e37uo190"})
    if head is not None:
        title = head.find("span")["title"]
        element_dict["title"] = title
    else:
        continue
    if location is not None:
        company_name = location.find("span", {"data-testid":"company-name" , "class": "css-63koeb eu4oa1w0"})
        if company_name is not None:
            element_dict["company"] = company_name.text
        city = location.find("div", {"data-testid":"text-location" , "class": "css-1p0sjhy eu4oa1w0"})
        if city is not None:
            element_dict["city"] = city.text
    metadata = element.find("div", {"class":"heading6 tapItem-gutter metadataContainer css-z5ecg7 eu4oa1w0"})
    if metadata is not None:
        salary_block = metadata.find("div", {"class":"metadata salary-snippet-container css-5zy3wz eu4oa1w0"})
        if salary_block is not None:
            salary = salary_block.find("div", {"data-testid":"attribute_snippet_testid", "class":"css-1cvvo1b eu4oa1w0"})
            boundaries = salary.text.split("€")[:-1]
            frequency = salary.text.split(" ")[-1]
            frequency_map = {"mois":"mensuel", "an":"annuel"}
            element_dict["frequency"] = frequency_map.get(frequency, "autre")
            for boundary in boundaries:
                if "de" in boundary.lower():
                    element_dict["min_salary"] = "".join(filter(str.isdigit, boundary))
                elif "à" in boundary.lower():
                    element_dict["max_salary"] = "".join(filter(str.isdigit, boundary))

    url_job = "https://fr.indeed.com" + element.find("a", {"class":"jcs-JobTitle css-jspxzf eu4oa1w0"})["href"]
    print(url_job)
    if any(element_dict.values()):
        df = pd.concat([df, pd.DataFrame(element_dict, index=[0])], ignore_index=True)

text/html;charset=UTF-8
Error decompressing content: BrotliDecompress failed
https://fr.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0AeFz1TRGbZndU5O3vrlIQpmrPI6f0p_To2HA7_S61zsB6Z63CvvIkS9ppMBqAgXWSyypVUdaCBCxAr86Frw0_87R17oJdI7RLH--GgE-7mHfyjmxGsD5iNLFdhp1WtfUWsVsQQ8Y4NigNHBsg920k9yjeySN0yOAD1OcxOxdWYawLk48W_UcXsWkGQe-iBMuqrVad-7o7GjMIX46K6ld77RS-v4WVD3yXPJzvjGW_vPM1J9iDVsWwVuoxTg8bCuecqdvOczjRyjca5v3BLM6tsAR_J4bhfV-hw52R2Myiko8hpFG1Ko2eXHMEXh7PZFBnl6uvGkRZ4XM5btE7-sm4c8XCHP_ymNjkCMh8lYnMPvcswdu5whk-DOzssofUZUr-Li3JpsL3qXItjU6KcBcH7rH1S3lWYod80_8SdAO-mLoA0R3XBCS05rmNu9nW3u1s6UgaiUODobXaoX6nFF0Ujjs9osmW4vqfd5Kl1yM_v6RLgeTdjb8M-VyX7zvaagQ5ngBVaSCKj79idMLpTjIpy9A-5YCApjxm85egFMhGmetqeizdPfCXm8PI3MXzrEbm4DiAOHE-tPROAOfO2hYMCGiTd-sTo4ezV0cJd5q7Yc-oINNyWZrObsKsoOBG-muYZJ6aXPiAThbeHgC1LHTWpDBQO7byqnPeV949F-Drgrv-utXMCXKkXSBvf_PuIvEiD0l2GYF1aXmMlMexsEvNvaxPV3QovbTZzpDznRzKBoL_g-twajLo8YmCuWmruIhCGx8gWB-NL8B0rp99wYYdp&xkcb=SoAw6_M3A2cDXGQHN50LbzkdCdPP&camk=nUmJqO2E8rjvaRm-8HjxdQ==&p=0&fvj=1&vjs=3
h

In [6]:
df.to_csv("data.csv", index=False, sep=";", encoding="utf-8")

In [10]:
element

<li class="css-5lfssm eu4oa1w0"><!--$--><div class="cardOutline tapItem dd-privacy-allow result job_a8178647cb7a8e3d resultWithShelf sponTapItem desktop css-yria2i eu4oa1w0"><div class="slider_container css-12igfu2 eu4oa1w0" data-testid="slider_container" dir="auto"><div class="slider_list css-ltdjbe eu4oa1w0"><div class="slider_item css-mk9n32 eu4oa1w0" data-testid="slider_item"><div class="job_seen_beacon"><table cellpadding="0" cellspacing="0" class="big6_visualChanges css-1v79ar eu4oa1w0" role="presentation"><tbody><tr><td class="resultContent css-1qwrrf0 eu4oa1w0"><div class="css-dekpa e37uo190"><h2 class="jobTitle css-198pbd eu4oa1w0" tabindex="-1"><a aria-label="tous les détails sur « Data Engineer - Alternance (F/H/NB) »" class="jcs-JobTitle css-jspxzf eu4oa1w0" data-hide-spinner="true" data-hiring-event="false" data-jk="a8178647cb7a8e3d" data-mobtk="1hvm9jaqaiem582v" href="/rc/clk?jk=a8178647cb7a8e3d&amp;bb=LmZvKr7gedvhmVdf2HrEmPakqs2Z79Wfw3s8YDcTjMSDDEGI_8LlKpsLtMb12-h3ERcGyG